In [5]:
import numpy as np
import pandas as pd

import os
folder_path = r"C:\VISSIM_Workspace\test\테스트"
mer_file = r"화성~서울(유고)_260108_003.mer"

start_interval = 1800
end_interval = 8400

with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

                # 가감속 차로 부분 검지기 제거
                original_df = original_df[~original_df["Measurem."].between(30000, 39999)]



                display(original_df[(original_df["TimeGroup"] == "6900~7200") & original_df["New_Measurement"] == 126])

,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],VehLength[m],,New_Measurement,TimeGroup
1639568,10008,6900.05,-1.0,16334,100,0,74.9,3.75,NaN,8,6900~7200
1639569,10012,6900.05,-1.0,16304,100,0,81.9,4.21,NaN,12,6900~7200
1639570,10025,6900.01,-1.0,16100,300,0,62.9,12.40,NaN,25,6900~7200
1639571,10092,6900.00,-1.0,15381,100,0,96.2,4.01,NaN,92,6900~7200
1639572,10101,6900.07,-1.0,15210,100,0,99.7,4.64,NaN,101,6900~7200
...,...,...,...,...,...,...,...,...,...,...,...
1685424,20055,7199.91,-1.0,16502,300,0,72.8,12.40,NaN,55,6900~7200
1685425,20064,7199.98,-1.0,16449,100,0,98.9,4.21,NaN,64,6900~7200
1685426,20276,7199.98,-1.0,17000,100,0,105.5,4.61,NaN,276,6900~7200
1685427,20483,7199.90,-1.0,16482,100,0,108.7,4.76,NaN,483,6900~7200


,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],VehLength[m],,New_Measurement,TimeGroup
